In [1]:
import cv2
import numpy as np
import time

In [4]:
!sudo apt-get update

Hit:1 http://raspbian.raspberrypi.org/raspbian bullseye InRelease
Hit:2 http://archive.raspberrypi.org/debian bullseye InRelease
Reading package lists... Done


In [5]:
!sudo apt-get install build-essential cmake pkg-config libjpeg-dev libtiff5-dev libjasper-dev libpng-dev libavcodec-dev libavformat-dev libswscale-dev libv4l-dev libxvidcore-dev libx264-dev libfontconfig1-dev libcairo2-dev libgdk-pixbuf2.0-dev libpango1.0-dev libgtk2.0-dev libgtk-3-dev libatlas-base-dev gfortran libhdf5-dev libhdf5-serial-dev libhdf5-103 python3-pyqt5 python3-dev -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libhdf5-dev' instead of 'libhdf5-serial-dev'
build-essential is already the newest version (12.9).
libpng-dev is already the newest version (1.6.37-3).
libpng-dev set to manually installed.
pkg-config is already the newest version (0.29.2-1).
python3-dev is already the newest version (3.9.2-3).
python3-dev set to manually installed.
python3-pyqt5 is already the newest version (5.15.2+dfsg-3).
python3-pyqt5 set to manually installed.
The following package was automatically installed and is no longer required:
  libfuse2
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  autoconf automake autopoint autotools-dev cmake-data debhelper dh-autoreconf
  dh-strip-nondeterminism dwz gettext gfortran-10 gir1.2-atspi-2.0
  gir1.2-gtk-2.0 hdf5-helpers icu-devtools intltool-debian libaec-dev libaec0
  libarchive-cpio-perl libarchive-zip-perl l

In [7]:
!sudo apt autoremove -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be REMOVED:
  libfuse2
0 upgraded, 0 newly installed, 1 to remove and 0 not upgraded.
After this operation, 288 kB disk space will be freed.

(Reading database ... 117174 files and directories currently installed.)
Removing libfuse2:armhf (2.9.9-5) ...
7Progress: [  0%] [..........................................................] 87Progress: [ 33%] [###################.......................................] 87Progress: [ 67%] [######################################....................] 8Processing triggers for libc-bin (2.31-13+rpt2+rpi1+deb11u7) ...

78

In [13]:
!pip install -U numpy

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 1.1 MB/s eta 0:00:0000:0100:010m


In [3]:
%cd ../

/home/jp_zuniga


/usr/local/lib/python3.9/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
%cd Yolo

/home/jp_zuniga/Yolo


/usr/local/lib/python3.9/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
net = cv2.dnn.readNet("yolov3-tiny-custom_best (4).weights", "yolov3-tiny-custom.cfg")
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [8]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

TypeError: only integer scalar arrays can be converted to a scalar index

In [9]:
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [10]:
cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
font = cv2.FONT_HERSHEY_PLAIN
frame_id = 1
if not cap.isOpened():
    raise ValueError("Error: Video capture not initialized.")

[ WARN:0] global /tmp/pip-wheel-ulre47vq/opencv-contrib-python_edf6b92c2aed48ceb1cae55c8d5542a6/opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


ValueError: Error: Video capture not initialized.

In [22]:

while True:
    starting_time = time.time()
    _, frame = cap.read()
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.0:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 1, color, 2)  

    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 4, (0, 0, 0), 3)
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    #frame_id = frame_id + 1
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [23]:
cap.release()
cv2.destroyAllWindows()